## Аугментируем данные через LLM и YandexTranslate

#### ***Deepseek way***

In [1]:
import pandas as pd
from openai import OpenAI
import re
import requests
from keys import deepkey, api_key, folder_id
import time

In [ ]:
# Создаем датафрейм с примерами для промта

example_df = pd.read_csv("data/data_en.csv")  

# Создаем датафрейм для аугментированных данных

df_augmentation = pd.DataFrame()


In [ ]:
# Создаем клиент для API

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=deepkey,
)

In [ ]:
# Промтим, отправляем запрос и запихиваем пары русский-английский в датасет

for i in range(100):
    examples = "\n".join(
        f"- {row['Russian']} — {row['English']}" for _, row in df.sample(15).iterrows()
    )

    prompt = (
        f"Вот примеры позиций меню на русском с переводом на английский:\n"
        f"{examples}\n\n"
        f"Скомбинируй 2 позиций меню в таком же формате (русский — английский). Они обязательно должны быть похожими на пример. Используй слова только из примеров! Никаких слов не из примера быть не должно, это обязательное условие!"
    )

    try:
        response = client.chat.completions.create(
            extra_headers={
                "HTTP-Referer": "https://yourdomain.com",
                "X-Title": "Menu Generator",
            },
            model="openai/gpt-4o-mini",
            messages=[
                {"role": "user", "content": prompt}
            ]
        )

        output = response.choices[0].message.content.strip()

        new_lines = [line.strip("- ").strip() for line in output.split("\n") if "—" in line]
        new_pairs = [tuple(part.strip() for part in line.split("—", 1)) for line in new_lines]

        result_df = pd.DataFrame(new_pairs, columns=["Russian", "English"])
        df_augmentation = pd.concat([df_augmentation, result_df], ignore_index=True)
        time.sleep(3)
        
        

    except Exception as e:
        print("Ошибка при генерации:", e)
        time.sleep(3)

df_augmentation

,Russian,English
0,1. Лосось на гриле с апельсиновым соусом и киноа,Grilled salmon with orange sauce and quinoa
1,2. Куриное филе с черным рисом и грибами шиитаке,Chicken fillet with black rice and shiitake mu...
2,3. Овощной рататуй с брокколи и перцем,Vegetable ratatouille with broccoli and peppers
3,4. Прошутто с дыней и вялеными томатами,Prosciutto with melon and sun-dried tomatoes
4,5. Тартар из говядины с кедровыми орехами и со...,Beef tartare with pine nuts and tartar sauce
...,...,...
993,"Таймырский олень с глазированнной свеклой, айо...","Taimyr deer with glazed beets, wild garlic and..."
994,Котлета из дикого кабана с хрустящими баклажан...,Wild boar cutlet with crispy eggplant and mari...
995,Цыплёнок в глазури из трюфельного мёда с печен...,Chicken in truffle honey glaze with baked pota...
996,Брускетта с печёным баклажаном и хрустящими ка...,Bruschetta with baked eggplant and crispy rock...


In [ ]:
# Чистим от LLM форматирования и сохраняем в CSV

df_augmentation['Russian'] = df_augmentation['Russian'].str.replace(r'^\d+\.\s*', '', regex=True)
df_augmentation['Russian'] = df_augmentation['Russian'].str.replace(r'[**]', '', regex=True)
df_augmentation['English'] = df_augmentation['English'].str.replace(r'[**]', '', regex=True)
df_augmentation.to_csv('data/big_deep_clean.csv')

#### ***Yandex Translate way***

In [ ]:
# Функция отправляет запрос в YandexTranslate и получает ответ

def translate(folder, api, language, text):
    body = {
        "folderId": folder,
        "texts": text,
        "targetLanguageCode": language
    }
    headers = {
    "Content-Type": "application/json",
    "Authorization": f"Api-Key {api}"}
    
    response = requests.post(
    "https://translate.api.cloud.yandex.net/translate/v2/translate",
    json=body,
    headers=headers,
)
    return response.json()

In [ ]:
# Берем данные для Яндекса
data_ya = pd.read_csv('data/cleaned_data.csv')
data_ya["English"][:10]

0     Bruschetta with ricotta cheese, avocado cream...
1     Panzanella salad with stracciatella di bufal,...
2     Green salad with kiwi, eggplant, cucumber, pe...
3     Salmon and dorado crudo, tomato salsa and capers
4     Orecchiette pasta, pistachio crème-sauce, tig...
5     Mezzi pasta, snow crab, neapolitan sauce and ...
6     Chicken breast with ptitim and truffle Parmes...
7     Tonno e cipolla pizza with tuna, red onion, o...
8     Pera picante pizza with dorblu cheese, pear, ...
9     Lemon pie with mascarpone cream, basil ice cream
Name: English, dtype: object

In [ ]:
# Переводим

for i in range(10):
    response = translate(folder_id, api_key, 'ru', data_ya["English"][i])
    print(f"{data_ya['English'].iloc[i]} - {response['translations'][0]['text']}")

 Bruschetta with ricotta cheese, avocado cream, lettuce and sunflower seeds -  Брускетта с сыром рикотта, кремом из авокадо, листьями салата и семечками подсолнечника
 Panzanella salad with stracciatella di bufal, olives, cheese croutons and tuna in its own juice -  Салат "Панцанелла" со страччателлой ди буфаль, оливками, сырными гренками и тунцом в собственном соку
 Green salad with kiwi, eggplant, cucumber, pees, spinach and citronette dressing -  Зеленый салат с киви, баклажанами, огурцами, горошком, шпинатом и цитрусовой заправкой
 Salmon and dorado crudo, tomato salsa and capers -  Крудо с лососем и дорадо, томатной сальсой и каперсами
 Orecchiette pasta, pistachio crème-sauce, tiger prawns and fried pistachio nuts -  Паста орекьетт с фисташковым крем-соусом, тигровыми креветками и жареными фисташковыми орешками
 Mezzi pasta, snow crab, neapolitan sauce and home-made stracciatella di bufala  -  Макаронные изделия, Снежный краб, соус neapolitan и домашняя Мурашка буйвола 
 Chicken 

#### Результаты аугментации с помощью яндекс переводчика мне не понравились, поэтому я решил не использовать его, остановив своё внимание на аугментации с помощью LLM

------------------------------------

#### Чистим аугментированные данные, смотрим на них поближе, совмещаем с исходным датасетом:

In [ ]:
# Открываем source data

data_en_source = pd.read_csv("data/data_en.csv").drop('Unnamed: 0', axis=1)

In [ ]:
# Совмещаем source и augmented
full_data = pd.concat([df_augmentation, data_en_source], axis=0, ignore_index=True)
full_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1310 entries, 0 to 1309
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Russian  1310 non-null   object
 1   English  1310 non-null   object
dtypes: object(2)
memory usage: 20.6+ KB


In [ ]:
# Удаляем дубликаты

full_data = full_data.drop_duplicates(keep="first")
full_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1301 entries, 0 to 1309
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Russian  1301 non-null   object
 1   English  1301 non-null   object
dtypes: object(2)
memory usage: 30.5+ KB


In [ ]:
# Сохраняем файл
full_data.to_csv("data/full_data_ver_2.csv")

### Как можно заметить по данным и коду, я прогонял его не один раз и всё пришло к тому, что у меня есть два датасета - версия 1 и версия 2. 

#### Они отличаются объемом и последующей чисткой. Над версией 2 я провел дополнительный анализ по количественным показателям -> **data_analysis**